In [2]:
from glyze import Glyceride
import numpy as np

c:\Users\oaksh\Desktop\GLYZE\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#Deodorization
Now, we will take the output mixture and simulate a short-path distillation process.
We will assume the deodorization process removes all of the unbinded fatty acids, so we are only interested in how it changes the glyceride mixture.

The DeodorizaztionSim class will take in a list of glycerides - a list of glycerides, initial_mols - the list of the initial quantity of each glyceride species, temperature - the temperature of the reactor, pressure - the pressure of the reactor, and residence_time - the residence time in the reactor. dt and efficiency factor can be tweaked for accuracy.

In [ ]:
import numpy as np
from typing import List, Dict

class DeodorizationSim:
    """
    Simulates the deodorization process for glycerides under vacuum conditions.
    
    Uses the Hertz-Knudsen equation to model evaporation rates based on 
    temperature, pressure, and molecular properties.
    """
    
    # Physical constants
    R_GAS_KJ = 8.314e-3  # kJ/(mol·K)
    R_GAS_J = 8.314      # J/(mol·K)
    
    def __init__(
        self, 
        glycerides: List, 
        initial_mols: np.ndarray, 
        temperature: float, 
        pressure: float, 
        residence_time: float, 
        dt: float = 1.0, 
        efficiency_factor: float = 0.5
    ):
        """
        Initialize the deodorization simulation.
        
        Args:
            glycerides: List of glyceride objects with properties (T_1mTorr_vapor_pressure, 
                       enthalpy_of_vaporization, molar_mass)
            initial_mols: Array of initial moles for each glyceride
            temperature: Process temperature (K)
            pressure: Process pressure (mTorr)
            residence_time: Total residence time (s)
            dt: Time step for simulation (s)
            efficiency_factor: Efficiency correction factor (0-1)
        """
        self.glycerides = glycerides
        self.mols = np.array(initial_mols, dtype=float)
        self.temperature = temperature
        self.pressure = pressure
        self.residence_time = residence_time
        self.dt = dt
        self.efficiency_factor = efficiency_factor
        
        # Validate inputs
        self._validate_inputs()
        
    def _validate_inputs(self):
        """Validate simulation parameters."""
        if len(self.glycerides) != len(self.mols):
            raise ValueError("Number of glycerides must match number of mole values")
        if self.temperature <= 0:
            raise ValueError("Temperature must be positive")
        if self.pressure < 0:
            raise ValueError("Pressure cannot be negative")
        if not 0 <= self.efficiency_factor <= 1:
            raise ValueError("Efficiency factor must be between 0 and 1")
        
    def simulate(self) -> np.ndarray:
        """
        Run the deodorization simulation.
        
        Returns:
            Final mole distribution after deodorization
        """
        print(f"Starting deodorization simulation:")
        print(f"  Total mols: {sum(self.mols):.4f}")
        print(f"  Temperature: {self.temperature} K")
        print(f"  Pressure: {self.pressure} mTorr")
        print(f"  Residence time: {self.residence_time} s")
        
        num_steps = int(self.residence_time / self.dt)
        
        for _ in range(num_steps):
            for idx, glyceride in enumerate(self.glycerides):
                evaporation_rate = self.calculate_evaporation_rate(glyceride)
                # Prevent negative moles
                delta_mols = evaporation_rate * self.dt * self.efficiency_factor
                self.mols[idx] = max(0, self.mols[idx] - delta_mols)
        
        print(f"Simulation complete. Final total mols: {sum(self.mols):.4f}")
        return self.mols
    
    def mol_fraction(self, glyceride) -> float:
        """
        Calculate the mole fraction of a specific glyceride.
        
        Args:
            glyceride: The glyceride to calculate mole fraction for
            
        Returns:
            Mole fraction (0-1)
        """
        total_mols = sum(self.mols)
        if total_mols == 0:
            return 0.0
        
        idx = self.glycerides.index(glyceride)
        return self.mols[idx] / total_mols
    
    def vapor_pressure(self, glyceride) -> float:
        """
        Calculate vapor pressure using the Clausius-Clapeyron equation.
        
        Args:
            glyceride: The glyceride with vapor pressure properties
            
        Returns:
            Vapor pressure (mTorr)
        """
        T_ref = glyceride.T_1uTorr_vapor_pressure
        delta_H_vap = glyceride.enthalpy_of_vaporization
        
        # Clausius-Clapeyron: ln(P2/P1) = -ΔH_vap/R * (1/T2 - 1/T1)
        vapor_pressure = np.exp(
            -delta_H_vap / self.R_GAS_KJ * (1/self.temperature - 1/T_ref)
        )
        
        return vapor_pressure  # mTorr
    
    def calculate_evaporation_rate(self, glyceride) -> float:
        """
        Calculate evaporation rate using the Hertz-Knudsen equation.
        
        Args:
            glyceride: The glyceride to calculate evaporation rate for
            
        Returns:
            Evaporation rate (mol/(m²·s))
        """
        x = self.mol_fraction(glyceride)
        P_vap = self.vapor_pressure(glyceride)
        M = glyceride.molar_mass
        
        # Hertz-Knudsen equation
        # Note: This gives mol/(m²·s) - needs surface area for total rate
        evaporation_rate = (x * P_vap) / np.sqrt(
            2 * np.pi * self.R_GAS_J * M * self.temperature
        )
        
        return evaporation_rate
    
    def get_results_summary(self) -> Dict:
        """
        Get a summary of the current simulation state.
        
        Returns:
            Dictionary with mole distribution and fractions
        """
        return {
            'total_mols': sum(self.mols),
            'mole_amounts': self.mols.copy(),
            'mole_fractions': [self.mol_fraction(g) for g in self.glycerides],
            'glyceride_names': [g.__class__.__name__ for g in self.glycerides]
        }

Now, I will instantiate the class and run a simulation

In [4]:
glyceride_names = ['G_N02D00_N02D00_N02D00', 'G_N03D00_N03D00_N03D00', 'G_N04D00_N04D00_N04D00', 'G_N05D00_N05D00_N05D00', 'G_N06D00_N06D00_N06D00', 'G_N07D00_N07D00_N07D00', 'G_N08D00_N08D00_N08D00', 'G_N09D00_N09D00_N09D00', 'G_N10D00_N10D00_N10D00', 'G_N11D00_N11D00_N11D00', 'G_N12D00_N12D00_N12D00', 'G_N13D00_N13D00_N13D00', 'G_N14D00_N14D00_N14D00', 'G_N15D00_N15D00_N15D00', 'G_N16D00_N16D00_N16D00', 'G_N17D00_N17D00_N17D00', 'G_N18D00_N18D00_N18D00', 'G_N19D00_N19D00_N19D00', 'G_N20D00_N20D00_N20D00', 'G_N21D00_N21D00_N21D00', 'G_N22D00_N22D00_N22D00']
glycerides = [Glyceride.from_name(name) for name in glyceride_names]
initial_mols = [1.0 for glyceride in glycerides]
deodorizer = DeodorizationSim(glycerides, initial_mols, temperature = 773.15, pressure = 200, residence_time = 60*30)
final_mols = deodorizer.simulate()

Starting deodorization simulation:
  Total mols: 21.0000
  Temperature: 773.15 K
  Pressure: 200 mTorr
  Residence time: 1800 s


C:\Users\oaksh\AppData\Local\Temp\ipykernel_10144\2670498093.py:117: RuntimeWarning: overflow encountered in exp
  vapor_pressure = np.exp(
C:\Users\oaksh\AppData\Local\Temp\ipykernel_10144\2670498093.py:139: RuntimeWarning: invalid value encountered in scalar multiply
  evaporation_rate = (x * P_vap) / np.sqrt(


KeyboardInterrupt: 